In [ ]:
from transformers import T5Tokenizer, MT5ForConditionalGeneration

Загрузка токенизатора

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(
    "kavlab/review-t5",
    revision="1.0.0"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/640k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Загрузка модели

In [ ]:
model = MT5ForConditionalGeneration.from_pretrained(
    "kavlab/review-t5",
    revision="1.0.0"
)

config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/259M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Проверка модели

In [ ]:
import random

In [ ]:
def make_query(row):
    query = (
        f"<rubrics>: {', '.join(row['rubrics_list'])} | "
        f"<raiting>: {row['rating']} | "
        f"<keywords>: {', '.join(row['key_words'])}"
    )
    return query

In [ ]:
max_target_length = 512

test_list = [{'rubrics': ['информационный интернет сайт', 'агентство недвижимости'],
  'keywords': ['тюмень', 'облачные сервисы']},
 {'rubrics': ['гостиница'], 'keywords': ['релокейт']},
 {'rubrics': ['зоосалон', 'зоопарикмахерская', 'зоомагазин'],
  'keywords': ['грумер анна', 'гаджеты', 'здоровье']},
 {'rubrics': ['спортивный магазин',
   'спортивная одежда и обувь',
   'товары для отдыха и туризма'],
  'keywords': ['управление персоналом']},
 {'rubrics': ['магазин чая', 'магазин посуды'],
  'keywords': ['чай', 'доставка', 'курьерская доставка', 'почта россии']},
 {'rubrics': ['бар', 'паб'], 'keywords': ['бар', 'эльдорадо']},
 {'rubrics': ['ресторан'], 'keywords': ['пицца', 'морс', 'пицца']},
 {'rubrics': ['кафе', 'доставка еды и обедов', 'банкетный зал'],
  'keywords': ['бар', 'бармены', 'гаджеты', 'удалённая работа']},
 {'rubrics': ['стоматологическая клиника'],
  'keywords': ['стоматология', 'здоровье']},
 {'rubrics': ['магазин продуктов',
   'магазин хозтоваров и бытовой химии',
   'супермаркет'],
  'keywords': ['интернет-магазин', 'интернет-торговля', 'интернет-маркетинг']}]

random.seed(42)

random_values = random.sample(test_list, 5)

for rating in range(1, 6):

  for random_value in random_values:

      params = {
          "rubrics_list": random_value["rubrics"],
          "rating": rating,
          "key_words": random_value["keywords"],
      }
      query = make_query(params)

      input_ids = tokenizer(query, return_tensors="pt").input_ids

      outputs = model.generate(
          input_ids,
          max_length=max_target_length,
          do_sample=True,
          temperature=0.8,
          repetition_penalty=1.4,
          num_beams=4,
          top_k=30,
          top_p=0.9,
      )

      print(query)
      print(tokenizer.decode(outputs[0], skip_special_tokens=True))
      print()

<rubrics>: гостиница | <raiting>: 1 | <keywords>: релокейт
Отвратительная гостиница. В номерах грязно, мебель старая. Персонал грубый. Завтраки не вкусные.

<rubrics>: информационный интернет сайт, агентство недвижимости | <raiting>: 1 | <keywords>: тюмень, облачные сервисы
Ужасное обслуживание, некомпетентные сотрудники, которые не знают о своей недвижимости. Приехала в Тюмень и ничего не знают.

<rubrics>: магазин чая, магазин посуды | <raiting>: 1 | <keywords>: чай, доставка, курьерская доставка, почта россии
Отвратительный сервис Заказывала чай в данном магазине. Привезли чай через два дня, сказали, что привезут через пять дней. В итоге обещали, что чай будет готов, а не ждать. Второй раз заказ так и не привезли. В итоге отказались от чая, т.к. никто не позвонил и не предупредил.

<rubrics>: магазин продуктов, магазин хозтоваров и бытовой химии, супермаркет | <raiting>: 1 | <keywords>: интернет-магазин, интернет-торговля, интернет-маркетинг
Ужасный магазин, постоянно очереди на кас